<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer: a visible layer which receives input from dataset
### Hidden Layer: a layer after input layer. It is hidden as it lies inside the neuron network and performs its function but will not interact directly with dataset
### Output Layer: a final layer of the network. It will receive the processed data from hidden layers and yields the result prediction
### Neuron: or nodes. They receive input data and pass signals to the next layer of nodes if a certain threshold is reached
### Weight: takes inputs, processes data, and reports outputs
### Activation Function: modifies output values to transform it into a format that makes sense for our context
### Node Map: a visual diagram of the architecture or "topology" of a neural network
### Perceptron: The simplest form of neural networks which consists of 2 input and 1 output cells

## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?
### First, neuron in the input layer receive data from the dataset. After reaching a certain threshold, they transmit the signals to the nodes of the hidden layers via weights. Finally, the processed data will be transmit to the output layer which will produce the resulting prediction. Bias helps adjust the output along with the weighted sum of the inputs to the neuron

#### Your Answer Here

## Write your own perceptron code that can correctly classify (99.0% accuracy) a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [1]:
import pandas as pd
data = { 'x1': [0,1,0,1],
         'x2': [0,0,1,1],
         'y':  [1,1,1,0]
       }

df = pd.DataFrame.from_dict(data).astype('int')

In [2]:
##### Your Code Here #####
# Initialize inputs and assume correct outputs
import numpy as np
np.random.seed(812)

inputs = np.array([
    [0, 0, 1],
    [1, 0, 1],
    [0, 1, 1],
    [1, 1, 0]
])

crt_outputs = [[0], [1], [0], [1]]

In [3]:
# sigmoid and its derivative functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1 - sx)

In [4]:
# Initialize random weights for the three inputs
weights = 2 * np.random.random((3,1)) - 1
print(weights)

[[ 0.0099616 ]
 [ 0.21185521]
 [-0.08502562]]


In [5]:
# Randomize weights 10000 times
for iteration in range(10000):
    weighted_sum = np.dot(inputs, weights)
    activated_output = sigmoid(weighted_sum)
    error = crt_outputs - activated_output
    adjustments = error * sigmoid_derivative(weighted_sum)
    weights +=  np.dot(inputs.T, adjustments)


In [6]:
print("Weights after training")
print(weights)

print("Output after training")
print(activated_output)

Weights after training
[[ 8.99063966]
 [-1.39867562]
 [-4.40781241]]
Output after training
[[0.01203579]
 [0.98987706]
 [0.00299913]
 [0.99949572]]


## Implement your own Perceptron Class and use it to classify a binary dataset: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [7]:
diabetes = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')
diabetes[:10]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


Although neural networks can handle non-normalized data, scaling or normalizing your data will improve your neural network's learning speed. Try to apply the sklearn `MinMaxScaler` or `Normalizer` to your diabetes dataset. 

In [75]:
from sklearn.preprocessing import MinMaxScaler, Normalizer

feats = list(diabetes)[:-1]
target = list(diabetes)[-1:]

MMS = MinMaxScaler()

X = MMS.fit_transform(diabetes[feats])
y = diabetes[target]

In [77]:
# Train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [78]:
class Perceptron(object):
    
    def __init__(self, learning_rate = 0.01, n_iter = 10):
        self.lr = learning_rate
        self.n_iter = n_iter
        self.activation = self._unit_step_func
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        """Fit training data
        X : Training vectors, X.shape : [#samples, #features]
        y : Target values, y.shape : [#samples]
        """
        n_samples, n_features = X.shape
        
        #init parameters
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        y_ = np.array(y)
        
        for _ in range(self.n_iter):
            for idx, x_i in enumerate(X):
                
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_pred = self.activation(linear_output)
                
                # Perceptron update
                update = self.lr * (y_[idx] - y_pred)
                
                self.weights += update * x_i
                self.bias += update

       
    
    def predict(self, X):
        """Return class label after unit step"""
        linear_output = np.dot(X, self.weights) + self.bias
        y_pred = self.activation(linear_output)
        return y_pred
    
    def _unit_step_func(self, x):
        return np.where(x>=0, 1, 0)

In [79]:
# Instantiate the model
model = Perceptron(learning_rate=0.01, n_iter=10)
model.fit(X_train, y_train)

In [80]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
y_pred

array([0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [81]:
acc_score = accuracy_score(y_test, y_pred)
print(f'The score of this perceptron model is {acc_score}')

The score of this perceptron model is 0.7272727272727273


## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?